# Nome : VÍTOR CORRÊA SILVA
# MATRÍCULA : 2019027237
# PROVA - COEFICIENTE DE CONVECÇÃO

In [174]:
import numpy as np
import os
import pandas as pd
from typing import Dict

## Descrição do problema

Determine a temperatura uniforme de equilíbrio $T_{placa}$ de uma placa vertical (altura $3.0m$; comprimento $6.0m$). Uma de suas faces (e) é exposta ao ar ambiente $T_{\infty}=310K$ e recebe um fluxo prescrito $q_e”=800\frac{W}{m^2}$, enquanto a outra face (d) recebe um fluxo prescrito $q_d”=140\frac{W}{m^2}$.

## Propriedades do Ar 
Para alimentar o código iterativo que irá calcular o coeficiente convectivo do problema foram extraídos os valores do livro para as propriedades termofisicas e utilizando esses valores foram construídos interpoladores que tem como varíavel de controle o valor da temperatura de filme $T_F$

In [175]:
class air:
    def __init__(self, air_conf : str = os.path.join(os.getcwd(),'air_params_treated.parquet')) -> None:
        self.data = pd.read_parquet(air_conf)
        (self.data)
    def air_parameters(self, T_f : np.float128 ) -> Dict:
        out =  { key : 
                np.interp( x = T_f, xp = self.data['T'].to_list(), fp = self.data[key].to_list()) 
                for key in self.data.loc[:, self.data.columns != 'T'].columns}
        out['Tf'] = T_f
        return out

In [176]:
class Hc():
    def __init__(self) -> None:
        self.air_config : air = air()
        self.g : np.float128 = 9.8
    def __Beta__(self, air_params : Dict)->np.float128:
        return 1/air_params['Tf']
    
    def __Grashof__(self,air_params : np.float128, L: np.float128, T_s = 410, T_inf = 310)-> np.float128:
        return (self.g*self.__Beta__(air_params)*(T_s - T_inf)*np.float_power(L,3))/np.float_power(air_params['v'],2)
    
    def __Rayleigh__(self,air_params : np.float128, L: np.float128, T_s = 410, T_inf = 310):
        return self.__Grashof__(air_params, L, T_s, T_inf)*air_params['Pr']
    
    def Nusselt(self,air_params : np.float128, L: np.float128, T_s = 410, T_inf = 310):
        part_1 : np.float128 = np.float_power(0.387*self.__Rayleigh__(air_params, L, T_s, T_inf),1/6)
        aux : np.float128 = 1 + np.float_power(0.492/air_params['Pr'], 9/16)
        part_2 : np.float128 = np.float_power(aux, 8/27)
        return np.float_power(0.825 +(part_1/part_2), 2)
    
    def value(self,air_params : np.float128, L: np.float128, T_s = 410, T_inf = 310) -> np.float128:
        return (self.Nusselt(air_params, L, T_s, T_inf)*air_params['k'])/L

## Modelagem do Problema
### Balanço de Energia
Como descrito no problema, a temperatura de equílibrio uniforme foi solicitada, isso significa que a energia armazenada no sistema deve ser igual a zero. Somando isso ao fato que não foi explicitado no problema que a placa gera algum tipo de calor temos que o balanço de enrgia pode ser descrito da seguinte forma:
$$ E_{entra} - E_{sai} = 0$$
Sendo que:
$$ E_{entra} = q_{e}"\cdot A + q_{d}"\cdot A = \left(q_{e}" + q_{d}"\right)\cdot A $$
$$ E_{sai} = h_{c}\cdot A\left(T_{placa} - T_{\infty}\right) $$
Logo temos  que :
$$ \left(q_{e}" + q_{d}"\right)\cdot A - h_{c}\cdot A\left(T_{placa} - T_{\infty}\right) = 0 $$
Como o valor solicitado é a a temperatura da placa tem-se que :
$$ T_{placa} = \frac{q_{e}" + q_{d}"}{h_{c}} + T_{\infty} $$
Fazendo a substituição dos valores conhecidos temos que :
$$ T_{placa} = \frac{800 + 140}{h_{c}} + 310 $$
Temos então que o resultado pode ser descrito como sendo :
$$ T_{placa} = \frac{940}{h_{c}} + 310 $$
### Condição de Parada
A partir do que foi descrito acima podemos afirmar que o código deve parar quando a seguinte equação for satisfeita:
$$\left(q_{e}" + q_{d}"\right)\cdot A - h_{c}\cdot A\left(T_{placa} - T_{\infty}\right) = 0 $$
$$\left(q_{e}" + q_{d}"\right) - h_{c}\left(T_{placa} - T_{\infty}\right) = 0 $$
### Espaço de Busca
Para ser possível definir os valores aceitáveis de $T_{placa}$ é necessário avaliar a equação. Tendo em mente que $h_c$ sempre será um número positivo entre 0 e 1 e que todos os valores conhecidos são números reais e positivos já é possível afirmar algumas condições:
1. $T_{placa}$  deve ser maior que zero.
2. $T_{placa}$ deve ser maior de $T_{\infty}$.
3. $\left(q_{e}" + q_{d}"\right)$ e $h_{c}\left(T_{placa} - T_{\infty}\right)$ devem ser iguais
4. O resultado do modelo independe da área $A$ da placa, porém depende da altura da placa.

Isso quer dizer que o problema possui soluções para o intervalo: 
$$ T_{\infty} < T_{placa} < \infty $$

## Modelo do Problema
Com o contexto definido do problema é possível escrever o problema da seguinte forma :
$$ \min f(T_{Placa}) = \left(q_{e}" + q_{d}"\right) - h_{c}\left(T_{placa} - T_{\infty}\right) $$
Para :
$$ T_{Placa} > T_{\infty} $$
$$ T_{Placa} \neq T_{\infty} $$

In [177]:
def f_objective(Qe : np.float128, Qd : np.float128, Hc: np.float128, T_inf : np.float128, T_placa):
    return (Qe+Qd)-(Hc*(T_placa - T_inf))

def run(T_placa : np.float128):
    print(f"Para temperatura estimada da placa sendo {T_placa} K :")
    L = 3.0
    T_inf = 310
    Qe = 800
    Qd = 140
    T_f = (T_placa + T_inf)/2.0
    print(f"Parâmetros do ar para a temperatura de filme igual a : {T_f} K")
    air_params = air().air_parameters(T_f)
    for key in air_params.keys():
        if key != 'Tf':
            print(key,' = ',air_params[key])
    h_c = Hc().value(air_params=air_params, L = L,T_s = T_placa,T_inf = T_inf)
    print('hc = ', h_c)
    f = f_objective(Qe,Qd,h_c,T_inf,T_placa)
    print(f"f({T_placa}) = {f}")
    return h_c, f_objective(Qe,Qd,h_c,T_inf,T_placa)

In [178]:
T_inicial = 311
T_final = 1600
print("NOME : VÍTOR CORRÊA SILVA")
print("MATRÍCULA : 2019027237")
for i in range(0,100):
    print(f'\nIntervalo Avaliado : [{T_inicial},{T_final}]\n')
    T_med = (T_inicial + T_final)/2
    print('---------------Inicio---------------')
    hc_init, f_init = run(T_inicial)
    print('---------------Meio---------------')
    hc_meio, f_meio = run(T_med)
    print('---------------Fim---------------')
    hc_fim, f_fim = run(T_final)
    if 0 in (f_init,f_meio,f_fim):
        break
    if f_init > f_meio:
        cond_1 =  f_meio < 0 < f_init
    else:
        cond_1 = f_meio > 0 > f_init
    if f_fim > f_meio:
        cond_2 = f_meio < 0 < f_fim
    else:
        cond_2 = f_meio > 0 > f_fim
    if cond_1 and not cond_2:
        T_final = T_med
    elif cond_2 and not cond_1:
        T_inicial = T_med

NOME : VÍTOR CORRÊA SILVA
MATRÍCULA : 2019027237

Intervalo Avaliado : [311,1600]

---------------Inicio---------------
Para temperatura estimada da placa sendo 311 K :
Parâmetros do ar para a temperatura de filme igual a : 310.5 K
Ro  =  1.126456
Cp  =  1.00742
mu  =  1.89556e-05
v  =  1.6946299999999997e-05
k  =  0.027077
alpha  =  2.4053999999999997e-05
Pr  =  0.70553
hc =  6.2945557867959465
f(311) = 933.705444213204
---------------Meio---------------
Para temperatura estimada da placa sendo 955.5 K :
Parâmetros do ar para a temperatura de filme igual a : 632.75 K
Ro  =  0.551056
Cp  =  1.05886
mu  =  3.167385e-05
v  =  5.76156e-05
k  =  0.048734
alpha  =  8.3712e-05
Pr  =  0.688275
hc =  32.86782610445437
f(955.5) = -20276.181750425298
---------------Fim---------------
Para temperatura estimada da placa sendo 1600 K :
Parâmetros do ar para a temperatura de filme igual a : 955.0 K
Ro  =  0.36476
Cp  =  1.1320000000000001
mu  =  4.1261e-05
v  =  0.00011317
k  =  0.06454
alpha  =  0.

In [179]:
print("\n-------------------------SOLUÇÃO FINAL-------------------------\n")
if f_init == 0 or (f_init == np.min([f_init,f_meio,f_fim])):
    print(f"T_placa = {T_inicial} K")
    print(f'h_c = {hc_init}')
    print(f"f({T_inicial}) = {f_init}")
elif f_meio == 0 or (f_meio == np.min([f_init,f_meio,f_fim])):
    print(f"T_placa = {T_med} K")
    print(f'h_c = {hc_meio}')
    print(f"f({T_med}) = {f_meio}")
elif f_fim == 0 or (f_fim == np.min([f_init,f_meio,f_fim])):
    print(f"T_placa = {T_final} K")
    print(f'h_c = {hc_fim}')
    print(f"f({T_final}) = {f_fim}")


-------------------------SOLUÇÃO FINAL-------------------------

T_placa = 355.2372391455762 K
h_c = 20.779340599788203
f(355.2372391455762) = 0.0
